In [6]:
# importing libraries
import tkinter as tk
import speech_recognition as sr
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from PIL import Image, ImageTk
import pyttsx3
import threading
import time
import pandas as pd
import socket
import re

# Set up NLP
# nltk.download('punkt')
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

#declaing global variables
global order_status 
global new_time
new_time = []
order_status = 0
total_amount = 0
global responses
responses = []
global response_index
global response_index2
response_index = -1
response_index2 = response_index
bill_details = []
bill_details2 = []
global menu_status
menu_status = 0
global time_place


# Loading the Excel sheets into a DataFrame
dialogue_data = pd.read_excel("C:/Users/Rishi/Desktop/jarvis/dialogue_data.xlsx")
menu_price_data = pd.read_excel("C:/Users/Rishi/Desktop/jarvis/menu_price.xlsx")

# Convert column names to lowercase
menu_price_data = menu_price_data.rename(columns=str.lower)

# Convert values to lowercase
menu_price_data = menu_price_data.applymap(lambda x: str(x).lower())
# Convert the DataFrame to a dictionary
dialogue_dict = dialogue_data.set_index('command')['response'].to_dict()

menu = menu_price_data.set_index('item')['price'].to_dict()
description = menu_price_data.set_index('item')['description'].to_dict()

# Dialogue management

# defining greetings
def handle_greeting():
    return dialogue_dict['greeting']

# menu displaying 
def handle_menu_request():
    global menu_status
    menu_status = 1
    global menu_carousel
    # Create the menu carousel
    menu_carousel = MenuCarousel(window)

    return dialogue_dict['menu_request']

#defining menucarousal for sliding menu cards 
class MenuCarousel:
    def __init__(self, master):
        self.master = master
        self.menu_images = [
            "C:/Users/Rishi/Desktop/jarvis/2.jpg",
            "C:/Users/Rishi/Desktop/jarvis/3.jpg",
            "C:/Users/Rishi/Desktop/jarvis/4.jpg",
            "C:/Users/Rishi/Desktop/jarvis/5.jpg",
            "C:/Users/Rishi/Desktop/jarvis/6.jpg",
            "C:/Users/Rishi/Desktop/jarvis/7.jpg",
            "C:/Users/Rishi/Desktop/jarvis/1.jpg"
        ]
        self.current_image = 0
        
        self.carousel_frame = tk.Frame(self.master)
        self.carousel_frame.pack()
        self.carousel_frame.place(x=95, y=0)
        
        self.menu_image_label = tk.Label(self.carousel_frame)
        self.menu_image_label.pack()
        
        # Calculate desired button size based on frame size
        frame_width = self.carousel_frame.winfo_width()
        button_width = frame_width // 10  # Adjust the denominator to control the button size ratio
        button_height = button_width
        
        # Load previous and next button images and resize them
        prev_image = Image.open("C:/Users/Rishi/Desktop/jarvis/left_arrow2.jpg")
        prev_image = prev_image.resize((50,50))
        prev_image = ImageTk.PhotoImage(prev_image)
        
        next_image = Image.open("C:/Users/Rishi/Desktop/jarvis/right_arrow2.jpg")
        next_image = next_image.resize((50,50))
        next_image = ImageTk.PhotoImage(next_image)
        
        cancel_image = Image.open("C:/Users/Rishi/Desktop/jarvis/cancel_image3.jpg")
        cancel_image = cancel_image.resize((100,50))
        cancel_image = ImageTk.PhotoImage(cancel_image)
        
        self.prev_button = tk.Button(self.master, image=prev_image, command=self.show_prev_image, bd=0)#,bg="#10104E", fg="white")
        self.prev_button.image = prev_image
        self.prev_button.pack()
        self.prev_button.place(x=40, y=390)
        
        self.next_button = tk.Button(self.master, image=next_image, command=self.show_next_image, bd=0)# bg="#10104E", fg="white", bd=0,font=("Helvetica", 12))
        self.next_button.image = next_image
        self.next_button.pack()
        self.next_button.place(x=645, y=390)
        
        self.cancel_button = tk.Button(self.master, image = cancel_image, command = self.cancel_display,bd=0)
        self.cancel_button.image = cancel_image
        self.cancel_button.pack()
        self.cancel_button.place(x=340, y=700)
        # Store the PhotoImage object
        self.menu_image = None  
        
        self.show_current_image()
        
    def show_current_image(self):
        image_path = self.menu_images[self.current_image]
        image = Image.open(image_path)
        image = image.resize((540, 800))
        self.menu_image = ImageTk.PhotoImage(image)
        self.menu_image_label.configure(image=self.menu_image)
        
    def show_prev_image(self):
        self.current_image = (self.current_image - 1) % len(self.menu_images)
        self.show_current_image()
        
    def show_next_image(self):
        self.current_image = (self.current_image + 1) % len(self.menu_images)
        self.show_current_image()
    def cancel_display(self):
        global menu_status
        self.carousel_frame.destroy()
        self.prev_button.destroy()
        self.next_button.destroy()
        self.cancel_button.destroy()
        menu_status -= 1

#defining function to display item information
def handle_item_info(command):
    response = description[command]
    output_text.insert(tk.END, response)
    speak_response(response)

#defining function to take order from user
global confirmation_buttons
def handle_order(items):
    global order_status
    global send_items
    global confirmation_buttons
    total_amount2 = 0
    for item in items:
        item = item.lower()
        if item in menu:
            price = menu[item]
            bill_details2.append((item, price))  # Append the item and price to the bill details list
            total_amount2 += int(price)
        else:
            response = "I'm sorry, {} is not available on the menu.".format(item)
            items.remove(item)
            output_text.insert(tk.END, response)
            speak_response(response)
    if total_amount2 > 0:
        order_status = 5
        send_items = items
        confirmation_buttons = ConfirmationButtons(window, confirm_order, cancel_order)
        confirmation_buttons.create_buttons()
        items = ",".join(items)
        response = "Great choice! Your order is {}. Would you like to confirm your order?".format(items)
        output_text.insert(tk.END, response)
        speak_response(response)

class ConfirmationButtons:
    def __init__(self, window, confirm_func, cancel_func):
        self.window = window
        self.confirm_func = confirm_func
        self.cancel_func = cancel_func
        self.yes_button = None
        self.no_button = None
        
    def create_buttons(self):
        self.yes_button = tk.Button(self.window, text="Yes", width=10, command=self.confirm_func)
        self.yes_button.pack(side="left", padx=10)
        self.yes_button.place(relx=0.65, rely=0.55)
        
        self.no_button = tk.Button(self.window, text="No", width=10, command=self.cancel_func)
        self.no_button.pack(side="left", padx=10)
        self.no_button.place(relx=0.8, rely=0.55)
        
    def destroy_buttons(self):
        if self.yes_button:
            self.yes_button.destroy()
        if self.no_button:
            self.no_button.destroy()



def confirm_order():
    confirmation("yes")

def cancel_order():
    confirmation("no")

#function for processing the confirmation command from user
def confirmation(command):
    global confirmation_buttons
    # After some time or action, call the following method to destroy the buttons:
    confirmation_buttons.destroy_buttons()
    if "yes" in command or "yeah" in command or "confirm" in command or "haa" in command:
        handle_order_confirmation('yes',send_items)
    elif 'no' in command or 'dont want' in command or 'cancel' in command or 'remove' in command:
        handle_order_confirmation('no',send_items)
    else:
        order_status = 0
        process_command(command)

# function for order confirmation from user and sending it to chef
def handle_order_confirmation(choice, items):
    global order_status
    total_amount = 0
    if choice.lower() == 'yes':
        for item in items:
            price = int(menu[item])
            # Append the item and price to the bill details list
            bill_details.append((item, price))  
            total_amount += price
        # Convert the order details to a string
        order_details = items
        resp = "Waiting for chef confirmation "
        speak_response(resp)
        
        # Send the order details to the server and receive the response
        new_order = ','.join(order_details)
        my_thread = threading.Thread(target=send_order_details, args=(new_order,)).start()
        order_status = 0
    else:
        response = "Okay, please continue with your order."
        output_text.insert(tk.END, response)
        speak_response(response)
        order_status = 0
        
#sending order to the chef using tcp protocol
def send_order_details(order_details):
    try:
        
        # Create a socket object
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        # Define the server's IP address and port number
        server_address = ('192.168.39.54', 10000)

        # Connect to the server
        client_socket.connect(server_address)
    
        client_socket.send(order_details.encode())

        # Receive the response from the server
        response = client_socket.recv(1024).decode()
        # Close the socket connection
        client_socket.close()
    
        responses.append(response)
        response_received(responses)
    except:
        response = "The chef is not available . please wait for some time."
        output_text.insert(tk.END, response)
        speak_response(response)
def response_received(response):
    
    response = responses[0]
    responses.remove(responses[0])
    count = find_time(response)
        
    response2 = "Order confirmed! Your order will be served in {}. Enjoy your meal!".format(response)
        
    order_status = 0
        
    output_text.insert(tk.END, response2)
    speak_response(response2)
    new_time.append(timer(count,window))
    new_time[-1].start_countdown()
    
# defining class for creating a timer for order waiting time
class timer():
    
    def __init__(self,count,window):
        self.times = count
        self.window = window

    def start_countdown(self):
        self.remaining_time = int(self.times)
        # Create a Label widget to display the countdown
        self.countdown_label = tk.Label(self.window, text="Order will be ready in: 00:00", font=("Arial", 16))
        self.countdown_label.pack(pady=10)
        timer_thread = threading.Thread(target=self.update_countdown).start()

    def update_countdown(self):
        if self.remaining_time > 0:
            minutes = self.remaining_time // 60
            seconds = self.remaining_time % 60
            self.countdown_label.config(text=f"Order will be ready in: {minutes:02d}:{seconds:02d}")
            self.remaining_time -= 1
            self.countdown_label.after(1000, self.update_countdown)
        else:
            self.countdown_label.config(text="Order is ready sir.\n your order will be served shortly")  # Update label text
            self.countdown_label.after(3000, self.remove_label)

    def remove_label(self):
        # Remove the label from the window
        self.countdown_label.destroy()  
        
# defining function for finding waiting time from the string given
def find_time(new_word):
    numbers = re.findall(r'\d+',new_word)
    return (int(numbers[0])*60)

def order_cancelation(command):
    if 'yes' in command or 'yeah' in command or 'confirm' in command:
        for i in new_time:
            i.remove_label()
        
        response = "order cancelled ."
        output_text.insert(tk.END, response)
        speak_response(response)
    else:
        response = "ok sir. order not cancelled."
        output_text.insert(tk.END, response)
        speak_response(response)
#function for handling unknown request 
def handle_unknown():
    return "I'm sorry, I didn't understand that. Can you please rephrase?"


def handle_bill_request():
    total_amount = 0
    bill_description = "Bill Details:\n"
    for item, price in bill_details:
        bill_description += "{} - Rs.{}\n".format(item, price)
        total_amount += price
    bill_description += "Total Amount: Rs.{}".format(total_amount)
    return bill_description

global current_item
current_item=[]
def handle_conversation(command):
    for item in current_item:
        current_item.remove(item)
    global order_status
    k=0
    info = 0
    command = command.lower()
    is_order = []
    for item in menu :
        item = item.lower()
        if item in command and ('price' in command or 'prices' in command or 'cost' in command or 'rate' in command or 'money' in command) :
            response = "the price of {} is {} rupees".format(item,str(menu[item]))
            speak_response(response)
            current_item.append(item)
        elif item in command and 'order' not in command and 'available' not in command :
            handle_item_info(item)
            current_item.append(item)
            info = 5
        elif item in command and 'order' not in command and 'available' in command :
            response = "yes sir, {} is available in our restaurent you can order it.".format(item)
            output_text.insert(tk.END, response)
            speak_response(response)
            current_item.append(item)
        elif item in command and 'order' in command:
            is_order.append(item)
            current_item.append(item)
    if is_order :
        handle_order(is_order)
        current_item.append(item)

    if ('menu' in command or 'available' in command or 'food' in command or 'items' in command) and 'remove' not in command and len(current_item)==0 :
        response = handle_menu_request()
        output_text.insert(tk.END, response)
        speak_response(response)
    elif 'information' in command or 'info' in command or 'about' in command:
        for item in menu:
            if item in menu:
                temp1 = 5
        if temp1 == 0:
            response = "Please specify an item from the menu to place an order."
            output_text.insert(tk.END, response)
            speak_response(response)  
    elif len(current_item)!=0 :
        pass
    elif len(command)==0 :
        pass
        output_text.inser()
    elif 'no' in command:
        tokens = word_tokenize(command)
        tokens = [token for token in tokens if token not in stop_words]
        item = ' '.join(tokens)  # Combine tokens into a sequence of words

        if item in menu:
            handle_order_confirmation('no', item)
        else:
            response = "Please specify an item from the menu to continue with your order."
            output_text.insert(tk.END, response)
            speak_response(response)
    elif 'bill' in command or 'total bill' in command:
        bill_description = handle_bill_request()
        output_text.insert(tk.END, bill_description)
        speak_response(bill_description)
    elif 'order details' in command or 'food details' in command or 'previous orders' in command:
        o_d = ""
        for item,prices in bill_details:
            o_d = o_d + "\n" + str(item)
        response = "your order details are : {}".format(o_d)
        speak_response(response)
        
    elif 'good bye' in command or 'goodbye' in command:
        response = "Goodbye! Have a great day!"
        output_text.insert(tk.END, response)
        speak_response(response)
        window.after(3000, window.destroy)  # Close the window after 3 seconds
    elif "cancel" in command :
        response = "do you want to cancel the order sir?"
        output_text.insert(tk.END, response)
        speak_response(response)
        order_status = 10
    elif 'remove' in command or 'close' in command :
        global menu_carousel
        if menu_status == 1 :
            response = "Got it sir"
            output_text.insert(tk.END, response)
            speak_response(response)
            menu_carousel.cancel_display()
        else:
            response = "menu is already removed sir"
            output_text.insert(tk.END, response)
            speak_response(response)
    elif 'next' in command or 'forward' in command or 'more' in command :
        menu_carousel.show_next_image()
    elif 'previous' in command or 'backward' in command :
        menu_carousel.show_prev_image()
    elif info == 5 :
        pass
    else:
        chat1 = 0
        for i in dialogue_dict :
            if i in command:
                response = dialogue_dict[i]
                output_text.insert(tk.END, response)
                speak_response(response)
                chat1 = 5
        for j in command :
            if j in menu :
                handle_item_info(j)
                chat1 = 5
        if(chat1 == 0):
            response = handle_unknown()
            output_text.insert(tk.END, response)
            speak_response(response)
            

def process_command(command):
    handle_conversation(command)
    
def process_command2(command):
    confirmation(command)
    
def process_command3(command):
    order_cancelation(command)
    
# Speech recognition
def listen_command():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        listening_label.config(text="Listening...", fg="blue")  # Update label text and color
        window.update_idletasks()  # Update the GUI immediately
        audio = r.listen(source)

    try:
        listening_label.config(text="Recognizing...", fg="blue")  # Update label text and color
        window.update_idletasks()  # Update the GUI immediately
        command = r.recognize_google(audio)
        input_text.delete(0, tk.END)  # Clear previous input
        input_text.insert(tk.END, command)
        
        listening_thread = threading.Thread(target=submit_command).start()
    except sr.UnknownValueError:
        print("Could not understand audio")
    except sr.RequestError as e:
        print("Could not request results; {0}".format(e))
    finally:
        listening_label.config(text="")  # Clear the listening label


def speak_response(response):
    output_text.pack()
    output_text.delete(1.0, tk.END)  # Clear previous output
    input_text.delete(0, tk.END)

    def perform_speech():
        for i, char in enumerate(response):
            output_text.insert(tk.END, char)
            output_text.update_idletasks()  # Update the GUI immediately

        engine = pyttsx3.init()
        engine.say(response)
        engine.runAndWait()

    speech_thread = threading.Thread(target=perform_speech).start()


def submit_command(event=None):
    global order_status
    if(order_status == 0):
        print("order status 1")
        command = input_text.get()
        process_command(command)
    elif(order_status == 10):
        print("order satus 3 ")
        command = input_text.get()
        order_status = 0
        process_command3(command)
    else:
        print("order status 2")
        command = input_text.get()
        process_command2(command)


# Create GUI
window = tk.Tk()
window.title("Rishi Restaurant")
window.attributes('-fullscreen', True)  # Set the window to fullscreen

# Load and display background image
background_image = Image.open("jarvis.jpg")
background_photo = ImageTk.PhotoImage(background_image)
background_label = tk.Label(window, image=background_photo)
background_label.place(x=0, y=0, relwidth=1, relheight=1)

# Load the background images for input and output frames
input_bg_image = Image.open("C:/Users/Rishi/Desktop/jarvis/input_pic2.png")
output_bg_image = Image.open("C:/Users/Rishi/Desktop/jarvis/output_pic.png")


# Convert the resized images to PhotoImage objects
input_bg_photo = ImageTk.PhotoImage(input_bg_image)
output_bg_photo = ImageTk.PhotoImage(output_bg_image)

# Fonts
title_font = ("Helvetica", 28, "bold")
label_font = ("arial", 14)
input_font = ("arial", 14)
output_font = ("arial", 12)

# Input Frame
input_frame = tk.Frame(window, bd=0)
input_frame.pack(pady=50)
input_frame.place(relx=0.75, rely=0.2, anchor="n", relwidth=0.25, relheight=0.1)

# Create labels for the frames and set the background images
input_bg_label = tk.Label(input_frame, image=input_bg_photo)
input_bg_label.place(x=0, y=0, relwidth=1, relheight=1)

# Resize the background images to match the frame dimensions.0.5
input_bg_image = input_bg_image.resize((50,50))

# Create a transparent Entry widget
input_text = tk.Entry(input_frame, width=30, font=input_font, bg="#10104E", fg="white",highlightthickness=0,insertbackground="white")
input_text.pack(side=tk.LEFT)
input_text.bind("<Return>", submit_command)  # Bind the Return key event

listen_frame = tk.Frame(window, bd=5)
listen_frame.pack(pady=10)
listen_frame.place(relx=0.5, rely=0.1, relwidth=0.03, relheight=0.06, anchor="n")

# Load and resize the image
submit_image = Image.open("googlemic.png")
submit_image = submit_image.resize((58, 58))  # Resize the image
submit_photo = ImageTk.PhotoImage(submit_image)

# Create button with custom image
submit_button = tk.Button(listen_frame, image=submit_photo, command=listen_command, bd=0, relief="flat", bg="blue")
submit_button.image = submit_photo  # Save reference to prevent garbage collection
submit_button.pack(side=tk.LEFT)

# Output Frame
output_frame = tk.Frame(window, bd=0)
output_frame.pack(pady=80)
output_frame.place(relx=0.75, rely=0.4, anchor="n", relwidth=0.25, relheight=0.15)

output_bg_image = output_bg_image.resize((output_frame.winfo_width(), output_frame.winfo_height()))

output_bg_label = tk.Label(output_frame, image=output_bg_photo)
output_bg_label.place(x=0, y=0, relwidth=1, relheight=1)

output_text = tk.Text(output_frame, height=5, width=60, font=output_font,bg="#10104E", fg="white", wrap=tk.WORD)


# Listening Label
listening_label = tk.Label(window, text="", font=label_font)
listening_label.pack()


# Hide the menu card initially
# Call the handle_greeting function
def display_greeting():
    greeting_response = handle_greeting()
    output_text.insert(tk.END, greeting_response)
    speak_response(greeting_response)


# Start the greeting thread
greeting_thread = threading.Thread(target=display_greeting)
greeting_thread.start()

window.mainloop()


order status 1
order status 1
order status 1


Exception in thread Thread-52:
Traceback (most recent call last):
  File "C:\Users\Rishi\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\Rishi\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Rishi\AppData\Local\Temp\ipykernel_10776\391027145.py", line 517, in perform_speech
  File "C:\Users\Rishi\anaconda3\lib\site-packages\pyttsx3\engine.py", line 177, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


order status 1
order status 1
order status 1
order status 1
order status 1


Exception in thread Thread-60:
Traceback (most recent call last):
  File "C:\Users\Rishi\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\Rishi\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Rishi\AppData\Local\Temp\ipykernel_10776\391027145.py", line 517, in perform_speech
  File "C:\Users\Rishi\anaconda3\lib\site-packages\pyttsx3\engine.py", line 177, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


order status 1
order status 1
order status 1
order status 1
order status 1
order satus 3 
order status 1
order status 1
order status 1
order status 1
order status 1
order status 1
